In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
dataset = pd.read_csv(r'C:\Users\tarun\Desktop\Churn_Modelling.csv')

In [3]:
tf.__version__

'2.9.1'

In [4]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
X = dataset.iloc[:, 3:-1].values
y = dataset.iloc[:, -1].values

In [6]:
X

array([[619, 'France', 'Female', ..., 1, 1, 101348.88],
       [608, 'Spain', 'Female', ..., 0, 1, 112542.58],
       [502, 'France', 'Female', ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 'Female', ..., 0, 1, 42085.58],
       [772, 'Germany', 'Male', ..., 1, 0, 92888.52],
       [792, 'France', 'Female', ..., 1, 0, 38190.78]], dtype=object)

In [7]:
y

array([1, 0, 1, ..., 1, 1, 0], dtype=int64)

In [8]:
dataset['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [9]:
# converting male and female to 1 and 0

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,2] = le.fit_transform(X[:,2])

In [10]:
X

array([[619, 'France', 0, ..., 1, 1, 101348.88],
       [608, 'Spain', 0, ..., 0, 1, 112542.58],
       [502, 'France', 0, ..., 1, 0, 113931.57],
       ...,
       [709, 'France', 0, ..., 0, 1, 42085.58],
       [772, 'Germany', 1, ..., 1, 0, 92888.52],
       [792, 'France', 0, ..., 1, 0, 38190.78]], dtype=object)

In [11]:
#converting Geography into binary form - it has 3 values - use one hot coding( when there is no relation)

In [12]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])],remainder='passthrough')
X = np.array(ct.fit_transform(X))


# X

In [13]:
 from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [14]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [15]:
#building ANN


#adding the output layer

In [16]:
ann = tf.keras.models.Sequential()

In [17]:
#adding the input layer and first hidden layer
ann.add(tf.keras.layers.Dense(units=8,activation='relu'))

In [18]:
#adding the second hidden layer
ann.add(tf.keras.layers.Dense(units=8,activation='relu'))

In [19]:
#adding the output layer
ann.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))
#end layers can use sigmodial function because it gives better probability

In [20]:
#training the ann - first compile

ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [21]:
#training ANN on train set
#batch-size - is used to send no of data at once\
#epochs is used to send overall data in the training
ann.fit(X_train,y_train,batch_size=32, epochs=100)

Epoch 1/100
250/250 [==============================] - 2s 2ms/step - loss: 0.5732 - accuracy: 0.7250
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4662 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4323 - accuracy: 0.8035
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.4036 - accuracy: 0.8275
Epoch 5/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3840 - accuracy: 0.8399
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3711 - accuracy: 0.8465
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3638 - accuracy: 0.8491
Epoch 8/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3589 - accuracy: 0.8511
Epoch 9/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3552 - accuracy: 0.8535
Epoch 10/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3524 - accura

#using ann model to predict if customer will leave the bank or not
#[
Use our ANN model to predict if the customer with the following informations will leave the bank:

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: $ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: $ 50000

So, should we say goodbye to that customer?
]

In [22]:
#we are using sc.transform because we used transformed data for prediction
#always use double brackets  to input values prediction

print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]]))> 0.5)

1/1 [==============================] - 1s 543ms/step
[[False]]


In [23]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)


63/63 [==============================] - 0s 1ms/step


In [24]:
from sklearn.metrics import confusion_matrix,accuracy_score
cm = confusion_matrix(y_test,y_pred)
print(cm)

[[1514   81]
 [ 197  208]]


In [25]:
print(accuracy_score(y_test,y_pred))

0.861
